# Generates a table of reads and counts of edit sites (mutations) for each read.

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import pysam
import re
from Bio import SeqIO
from tqdm import tnrange
from collections import defaultdict

In [2]:
bam_file = '/projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/workflow_peakcaller/small_example_materials/peakcalling_outputs/window.bam'

In [3]:
def get_readcount(bam_file):
    """
    Parses a bam file idxstats to get the number of reads.
    The BAM file MUST have an index.
    """
    num_reads = pysam.idxstats(
        bam_file
    ).split('\n')
    nums = {}
    for num in num_reads:
        try:
            chrom, chrlen, mapped, unmapped = num.split('\t')
            nums[chrom] = int(mapped) + int(unmapped)
        except ValueError:
            print(num)
    return pd.DataFrame(nums, index=['num']).T.sum().values[0]

read_count = get_readcount(bam_file)
read_count

14397

In [9]:
test_read_name = 'K00180:915:H5LJKBBXY:4:1108:7984:5200_GCCAGACAAT'
# for isolating reads to test
bam = pysam.AlignmentFile(bam_file, "rb")
for read in bam:
    if read.query_name == test_read_name and not read.is_secondary:
        test_read = read

test_read

In [7]:
def get_left_softclip(read):
    """
    Gets the FIRST softclip instance of a read (we don't care about the last softclip.)
    """
    for t in read.cigartuples:
        if t[0] == 4:
            return t[1]
    return 0

edit_file = 'Alex_m1A.m1A-0MMS-N1.umi.r1.fq.genome-mappedSoSo.rmDupSo.txt.test'

if True: # not os.path.exists(edit_file): # only make a new one if we don't have the file.
    progress = tnrange(read_count)
    bam = pysam.AlignmentFile(bam_file, "rb")
    with open(edit_file, 'w') as o:
        for read in bam:
            if read.is_secondary or read.is_unmapped: # no pcr duplicates to worry about.
                pass
            # elif read.query_name == 'K00180:915:H5LJKBBXY:4:1104:14448:44992_AATCCATAGT':
            else:
                # if read.query_name == test_read_name:
                edit_count = 0                        # how many C>T edits
                read_seq = read.query_sequence        # original qseq
                md = read.get_tag("MD").upper()       # get MD tag for each read
                # print(md)
                # print(read.cigarstring)
                if read.is_reverse:
                    ref = 'G'
                    query = 'A'
                else:
                    ref = 'C'
                    query = 'T'
                mismatches_regex = "(\d+)([\^ATCG]+)"
                mismatches = re.findall(mismatches_regex, md)  # JUST finds mismatches and deletions (MD does not contain insertion info)

                # Get Insertions, junctions and softclips from CIGAR
                inserts = []    # Total insert [start positions, lengths]
                junctions = []  # Total junction [start positions, lengths]
                insert_pos = 0  # Running counter of (softclip-adjusted) insertion positions
                jxc_pos = 0     # Running counter of (softclip-adjusted) junction positions
                softclip_offset = get_left_softclip(read)
                read_seq = read_seq[softclip_offset:]
                
                # respect to the read, we want to convert CIGAR positions (ref) to read positions. 
                # So we don't care about ref positions in CIGAR, though we need to keep track of the insertion len
                # We also don't care about junction spans either? Since they aren't counted in the read anyways (but might be important if we modify this to count refbases)
                for t in read.cigartuples: # t[0] = modifier, t[1] = length of modified seq
                    if t[0] == 1:  # BAM_CINS (I)
                        inserts.append([insert_pos - softclip_offset, t[1]])  # CIGAR tuples all include softclip pos, but isn't counted anywhere else. Let's remove this offset from INSERTION positions
                    elif t[0] == 3:  # BAM_CREF_SKIP (N)
                        junctions.append([jxc_pos - softclip_offset, t[1]])   # CIGAR tuples all include softclip pos, but isn't counted anywhere else. Let's remove this offset from JUNCTION positions
                    if t[0] != 2 and t[0] != 3:  # BAM_CDEL (D), BAM_CREF_SKIP (N)
                        insert_pos += t[1]
                        jxc_pos += t[1]
                junctions = []
                # print(read.cigartuples)
                # print("old inserts", inserts)
                # Re-align the reference positions so it is with respect to the read sequence
                # while len(junctions) > 0:
                #     jpos = junctions.pop(0)[0]
                #     print("JPOS: {}".format(jpos))
                #     for i in inserts:
                #         if i[0] >= jpos:
                #             i[0] -= jpos
                # print("new inserts", inserts)
                # For every MD "mismatch" (SNP, deletion)
                if mismatches:
                    read_pos = 0                     # read position counter
                    # ref_pos = read.reference_start   # Don't think I actually use this..
                    for mismatch in mismatches:
                        ref_allele = mismatch[1]
                        # If an insertion is found between the current position and the next position, adjust the current read position (read_pos)
                        try:
                            # if read.query_name == test_read_name:
                            #     print("current read pos: {}, mismatch offset: {}, insert pos: {}".format(read_pos, mismatch[0], inserts[0][0]))
                            while read_pos + int(mismatch[0]) >= inserts[0][0]:
                                # if read.query_name == test_read_name:
                                #     print("inside while. There is an insertion between {} and {} (insert pos {})".format(read_pos, read_pos + int(mismatch[0]), inserts[0][0]))
                                #     print("inside while.. current read pos: {}, mismatch offset: {}, insert pos: {}".format(read_pos, mismatch[0], inserts[0][0]))
                                #     print("(before pop) inserts, read pos: {}, insert pos: {}, insert size: {}".format(read_pos, inserts[0][0], inserts[0][1]))
                                
                                read_pos += inserts[0][1]
                                inserts.pop(0)
                                # print("INSERTS: {}".format(inserts))
                                # if read.query_name == test_read_name:
                                #     print("(after pop) inserts, read pos: {}, insert pos: {}, insert size: {}".format(read_pos, inserts[0][0], inserts[0][1]))

                        except IndexError:
                            pass
                        read_pos += int(mismatch[0])
                        # ref_pos += int(mismatch[0])

                        # Test read to make sure we're capturing what IGV shows. 
                        # if read.query_name == test_read_name:
                        print("MISMATCH: {}, Current read pos: {} and ref: {} and read: {} (flanked by: {})".format(read.query_name, read_pos, ref_allele, read_seq[read_pos], read_seq[read_pos-10:read_pos] + '[' + read_seq[read_pos] + ']' + read_seq[read_pos+1:read_pos+10]))

                        read_allele = read_seq[read_pos]

                        # increment by one (except in the case of deletion, we won't increment read_pos)
                        if not ref_allele.startswith('^'):
                            read_pos += 1

                        # if read.query_name == 'cf39e8e1-6e49-4d4f-830a-a2a639816232':
                        #     print("read_pos: {}, ref: {}, read: {}".format(read_pos, ref_allele, read_allele))

                        if ref_allele == ref and read_allele == query:
                            edit_count += 1

                o.write("{}\t{}\n".format(read.query_name, edit_count))
            progress.update(1)

MISMATCH: A00475:396:HW7FGDRXY:1:1145:8938:10989, Current read pos: 48 and ref: C and read: T (flanked by: TTTATATTCT[T]ATCCTACCA)
MISMATCH: A00475:396:HW7FGDRXY:1:1143:4291:19883, Current read pos: 37 and ref: C and read: T (flanked by: TTTATATTCT[T]ATCCTACCA)
MISMATCH: A00475:396:HW7FGDRXY:1:1143:4291:19883, Current read pos: 88 and ref: G and read: A (flanked by: ACTACTCCGG[A]AAAAAAAGA)
MISMATCH: A00475:396:HW7FGDRXY:1:1116:22688:28761, Current read pos: 28 and ref: C and read: T (flanked by: TTTATATTCT[T]ATCCTACCA)
MISMATCH: A00475:396:HW7FGDRXY:1:1116:22688:28761, Current read pos: 79 and ref: G and read: A (flanked by: ACTACTCCGG[A]AAAAAAAGA)
MISMATCH: A00475:396:HW7FGDRXY:1:1139:30825:35149, Current read pos: 28 and ref: C and read: T (flanked by: TTTATATTCT[T]ATCCTACCA)
MISMATCH: A00475:396:HW7FGDRXY:1:1139:30825:35149, Current read pos: 79 and ref: G and read: A (flanked by: ACTACTCCGG[A]AAAAAAAGA)
MISMATCH: A00475:396:HW7FGDRXY:1:2228:31006:30420, Current read pos: 28 and ref

In [8]:
print("{}:{}-{}".format(test_read.reference_name, test_read.reference_start, test_read.reference_start + 50))

chr1:15191-15241
